In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
scales = [3]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    filter = (x > 20.0)
    filtered_x = s.where(filter, x)
    return s.mean(filtered_x)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:172: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [7]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



 <------------- Numerical Fidelity Report (input_scale: 3, param_scale: 3, scale_input_multiplier: 10) ------------->

+--------------+--------------+-----------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error   | median_error | max_error | min_error    | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+--------------+--------------+-----------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.016670227 | -0.033340454 | 0         | -0.033340454 | 0.016670227    | 0.033340454      | 0.033340454   | 0             | 0.00055579294      | -0.00030676697     | 0.00030676697          |
+--------------+--------------+-----------+--------------+----------------+------------------+---------

==== Generate & Calibrate Setting ====
scale:  [3]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":3,"param_scale":3,"scale_rebase_multiplier":10,"lookup_range":[-294,1278],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":3936,"total_assignments":911,"total_const_size":205,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,3],"model_input_scales":[3],"module_sizes":{"kzg":[],"poseidon":[3936,[1]]},"required_lookups":[{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709715528777}


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 0.5483167171478271 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 54.375
==== Generating Proof ====
proof:  {'instances': [['11e5950d0c875140b38d8b4bc0997697b7b183cfdbc19e767d87caf0020da12a', '0100000000000000000000000000000000000000000000000000000000000000', 'b301000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x2376a6c2ed0c2cf1a8660e22a9d8d5529054156c529a1739e4ccc598dc30dc5a1ad43912d8bcaaa1e093c9fb2963a7b0f2548e43c70435807ce93386c5eb1b8e28d831f7650e5fa128897eaf3c582314f6f6e9fefae283c4db76c472dff3c1252c124e56c41f07a5983d5fc41bb392715fd9f61e1e800fbbb97b5a826900718b1e2139cbccc462efd1d976a108f58d0189c6799ec7880ae4d89bcf3a416969f008af0dc41ac1214b464c05dfb73d19619475d271757930497ff1233856269fe92c3ee4eb18c6f6090eb90256b999afc3165942f257348162ae180623334e99ee13cd3ff6e43f07334f3b7c0489ce0256ae0e9fccc95ff478aeab91bb0a5203d428818854afa822477d781d9c417e50eb69fb2945ad6d85c16991ace18cb597bf113ab20e43b93

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [54.375]
